1. Instead of only extracting numerical words from sentences, the function "nummod_in_sentence" extracts nummod dependencies pair. For example, the result of "He has three girl friends." is "[[three, friends]]", and the result of "Three ugly men walked." is "[[three, men]]".

2. (To do) Addition examples may have the following features: a. numerical words modify the same object in both sentences, b. numerical word in hypothesis is larger than those in premise. 

3. Some examples are at the bottom.

In [108]:
import src.utilities.utils as utils 
test_data = utils.read_jsonl('data/snli_1.0/snli_1.0_test.jsonl')

In [97]:
# nums = ['2','3','4','5','6','7','8','9','10']
# special_numericals = ['some', 'many', 'all'] # TODO
# number_words = utils.plural_words_for_numbers + nums

convert_dict_plural = {
    'one':1,
    'two':2,
    'three':3,
    'four':4,
    'five':5,
    'six':6,
    'seven':7,
    'eight':8,
    'nine':9,
    'ten':10,
    '1':1,
    '2':2,
    '3':3,
    '4':4,
    '5':5,
    '6':6,
    '7':7,
    '8':8,
    '9':9,
    '10':10
}

import spacy
nlp = spacy.load('en')

def nummod_in_sentence(sentence):
    doc = nlp(sentence.lower())
    return [[token.text, token.head.text] for token in doc 
            if token.text in convert_dict_plural and token.dep_=='nummod']

def check_hyponym(hypo_word, word_h):
    #print(hypo_word, word)
    word0 = wn.synsets(hypo_word)
    word = wn.synsets(word_h)
    if len(word0)>= 1 and len(word)>=1:
        word0 = word0[0]
        word = word[0]
        hypowords = set([i for i in word.closure(lambda s:s.hyponyms())])
        return word0 in hypowords
    else:
        print("{} or {} is not found in wordnet".format(hypo_word, word_h))
        return False

def check_same_word(sent1_word, sent2_word):
    return sent1_word == went2_word

def check_potential_addition(sent1_num_words, sent2_num_words):
    # To do: len(sent2_num_words)>1
    if len(sent2_num_words)==1 and len(sent1_num_words)>len(sent2_num_words):
        for word_list in sent1_num_words:
            if word_list[1]!= sent1_num_words[0][1] and check_hyponym(word_list[1], sent2_num_words[0][1]) == False:
                return False
        return True
    return False

def subset_with_numericals(dataset):
    result = []
    for data in dataset:
        sent1_num_words = numerical_word_in_sentence(data['sentence1'])
        sent2_num_words = numerical_word_in_sentence(data['sentence2'])

        if sent1_num_words or sent2_num_words:
            result.append(data)
    return result

def subset_with_potential_addition(dataset):
    result = []
    for data in dataset:
        sent1_num_words = nummod_in_sentence(data['sentence1'])
        sent2_num_words = nummod_in_sentence(data['sentence2'])
        
        if check_potential_addition(sent1_num_words, sent2_num_words): # TODO: When len(list) > 1
            print((sent1_num_words, sent2_num_words))
#             print(data['gold_label'])
            print(data['sentence1'])
            print(data['sentence2'])
#             print()
            result.append(data)
    return result

def subset_by_relationship_type(dataset, relation):
    result = []
    return [data for data in dataset if data['gold_label'] == relation]

In [98]:
test_data_entailment = subset_by_relationship_type(test_data, 'entailment')

In [99]:
len(test_data_entailment)

3368

In [100]:
test_entailment_numericals = subset_with_potential_addition(test_data_entailment)

others or men is not found in wordnet
([['three', 'men'], ['two', 'men']], [['three', 'men']])
A skier being helped onto a red stretcher by three men, while two other men and a woman prepare to help.
Three men help a skier


In [101]:
a = "A skier being helped onto a red stretcher by three men, while two other men and a woman prepare to help."
doc = nlp(a)
for i, token in enumerate(doc):
    print(i, token.text, token.head.text, token.dep_)

0 A skier det
1 skier helped nsubjpass
2 being helped auxpass
3 helped helped ROOT
4 onto helped prep
5 a stretcher det
6 red stretcher amod
7 stretcher onto pobj
8 by helped agent
9 three men nummod
10 men by pobj
11 , helped punct
12 while prepare mark
13 two men nummod
14 other men amod
15 men prepare nsubj
16 and men cc
17 a woman det
18 woman men conj
19 prepare helped advcl
20 to help aux
21 help prepare xcomp
22 . helped punct
